# Maurice

## Read-in, prepare data & load functions

In [ ]:
#pip install seaborn
#!pip install statsmodels

In [1]:
import os 
import pandas as pd
import numpy as np
import h5py
import pickle as pkl

#import functions_read_data as rdat
# Tensorflow/Keras
import tensorflow as tf
import time
print(tf.__version__)
from tensorflow import keras
from tensorflow.keras import layers
import pickle as pkl

from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard

from tensorflow.keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator


2.2.0


Using TensorFlow backend.


In [2]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU'),
 PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
DIR = "/tf/notebooks/schnemau/xAI_stroke_3d/"
os.chdir(DIR)
print(os.getcwd())
IMG_DIR = "/tf/notebooks/hezo/stroke_perfusion/data/"
OUTPUT_DIR = "/tf/notebooks/schnemau/xAI_stroke_3d/data/"
path_img = IMG_DIR + 'dicom_2d_192x192x3_clean_interpolated_18_02_2021_preprocessed2.h5'
path_tab = IMG_DIR + 'baseline_data_zurich_prepared.csv'

/tf/notebooks/schnemau/xAI_stroke_3d


In [4]:
import functions_metrics as fm
import functions_model_definition as md
import functions_read_data as rdat
import Utils_maurice as utils

In [5]:
# Own functions
from functions.plot_slices import plot_slices
#ontram functions
from k_ontram_functions.ontram import ontram
from k_ontram_functions.ontram_loss import ontram_loss
from k_ontram_functions.ontram_metrics import ontram_acc, ontram_auc
from k_ontram_functions.ontram_predict import predict_ontram, get_parameters
from functions.augmentation3d import zoom, rotate, flip, shift

In [6]:
which_split = 6
(X_train, X_valid, X_test, X_tab_train, X_tab_valid, X_tab_test), (Y_train, Y_valid, Y_test), results = utils.read_and_split_img_data_andrea_maurice(
    path_img = IMG_DIR + 'dicom_2d_192x192x3_clean_interpolated_18_02_2021_preprocessed2.h5', 
    path_tab = IMG_DIR + 'baseline_data_zurich_prepared.csv', 
    path_splits = '/tf/notebooks/schnemau/xAI_stroke_3d/data/andrea_splits.csv', 
    split = which_split)

Y_train_MRS = to_categorical(Y_train)
Y_valid_MRS = to_categorical(Y_valid)
Y_test_MRS = to_categorical(Y_test)

#X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], X_train.shape[2], X_train.shape[3], 1))
#X_valid = X_valid.reshape((X_valid.shape[0], X_valid.shape[1], X_valid.shape[2], X_valid.shape[3], 1))
#X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], X_test.shape[2], X_test.shape[3], 1))

## ONTRAM-Set-Up

## Jonas-Set-Up

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_probability as tfp
from tensorflow.keras.models import Sequential, Model

In [ ]:
def stroke_binary_3d_softmax(input_dim = (128, 128, 28,1), 
                     output_dim = 2,
                     last_activation = "softmax"):
    
    initializer = keras.initializers.he_normal(seed = 2202)
    
    #input
    inputs = keras.Input(input_dim)
    
    # conv block 0
    x = layers.Convolution3D(32, kernel_size=(3, 3, 3), padding = 'same', activation = 'relu')(inputs)
    x = layers.MaxPooling3D(pool_size=(2, 2, 2))(x)
    
    # conv block 1
    x = layers.Convolution3D(32, kernel_size=(3, 3, 3), padding = 'same', activation = 'relu')(x)
    x = layers.MaxPooling3D(pool_size=(2, 2, 2))(x)
    
    # conv block 2
    x = layers.Convolution3D(64, kernel_size=(3, 3, 3), padding = 'same', activation = 'relu')(x)
    x = layers.MaxPooling3D(pool_size=(2, 2, 2))(x)
    
    # conv block 3
    x = layers.Convolution3D(64, kernel_size=(3, 3, 3), padding = 'same',activation = 'relu')(x)
    x = layers.MaxPooling3D(pool_size=(2, 2, 2))(x)
    
    #x = layers.Flatten()(x)
    x = layers.GlobalAveragePooling3D()(x) 
    # flat block
    x = layers.Dense(128, activation = 'relu')(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(128, activation = 'relu')(x)
    x = layers.Dropout(0.3)(x)
    
    out = layers.Dense(units=output_dim, activation = last_activation)(x) # softmax (output_dim must be at least 2)

    # Define the model.
    model_3d_softmax = Model(inputs=inputs, outputs=out)
    
    return model_3d_softmax




## Data-Prep for Training, Valid, Test

In [ ]:
C = Y_train_MRS.shape[1]
batch_size = 5
epochs = 250
ntrain = 325
nvalid = 40
ntest = 40

## CI_B Real no ensemble

In [ ]:
def validation_preprocessing(volume, label):
    """Process validation data by only adding a channel."""
    volume = tf.expand_dims(volume, axis=3)
    return volume, label

# Define data loaders
train_loader = tf.data.Dataset.from_tensor_slices((X_train[:ntrain], Y_train_MRS[:ntrain]))
validation_loader = tf.data.Dataset.from_tensor_slices((X_valid[:nvalid], Y_train_MRS[:nvalid]))

from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.15,
    shear_range=0.15,
    fill_mode="nearest")
datagen.fit(X_train[:ntrain])

validation_dataset = (
    validation_loader.shuffle(len(X_valid))
    .map(validation_preprocessing)
    .batch(batch_size)
    .prefetch(2)
)

In [ ]:
input_dim = np.expand_dims(X_train, axis = -1).shape[1:]
output_dim = 1

# call model
ontram_CI_B_real = ontram(img_model_linear(input_dim, output_dim))

#compile
ontram_CI_B_real.compile(optimizer=keras.optimizers.Adam(learning_rate=5*1e-5),
                        loss=ontram_loss(C, batch_size),
                        metrics=[ontram_acc(C, batch_size)])

checkpoint_cb = keras.callbacks.ModelCheckpoint(
    filepath = '/tf/notebooks/schnemau/xAI_stroke_3d/ensembling_results/real2.h5',
    verbose = 1,
    save_weights_only = True,
    monitor = "val_loss", #'val_acc',
    mode = 'min',
    save_best_only = True)

early_stopping_cb = keras.callbacks.EarlyStopping(monitor="val_loss", patience=100, restore_best_weights=True)

In [ ]:
train_CIB_real = False

In [ ]:
# Train the model, doing validation at the end of each epoch
epochs = 1

if train_CIB_real:
    hist = ontram_CI_B_real.fit(
        datagen.flow(X_train, Y_train_MRS, batch_size=batch_size, shuffle=True),
        validation_data=validation_dataset,
        epochs=epochs,
        shuffle=True,
        verbose=1,
        callbacks=[checkpoint_cb, early_stopping_cb]
)  

## CI_B no Real ensemble

In [ ]:
def validation_preprocessing(volume, label):
    """Process validation data by only adding a channel."""
    volume = tf.expand_dims(volume, axis=3)
    return volume, label

In [ ]:
id_tab_ens = pd.read_csv("/tf/notebooks/schnemau/xAI_stroke_3d/data/10Fold_ids_V0.csv", sep=",") # for V0
X_ens = np.load("/tf/notebooks/schnemau/xAI_stroke_3d/data/prepocessed_dicom_3d.npy")

In [ ]:
num_splits = 10   # 10 for all models
num_models = 5    # see overview of all models (above)

batch_size = 5    
epochs = 250      

In [ ]:
train_CIB_noreal = True 

In [ ]:
start0 = time.time()
for i in range(num_splits):
    start1 = time.time()
    print("\n\n\n\n################################################################################")
    print("Split " + str(i))
    print("################################################################################\n\n\n\n")
    
    (X_train, X_valid, X_test), (y_train, y_valid, y_test) = rdat.split_data(id_tab_ens, X_ens, i)
    
    y_train_enc = to_categorical(y_train)
    y_valid_enc = to_categorical(y_valid)
    y_test_enc = to_categorical(y_test)

    # loop over model instances (ensembling)
    for j in range(num_models):
        start2 = time.time()
        print("\n\n#######################################################")
        print("Split " + str(i) + " Model " + str(j))
        print("#######################################################\n\n")       

        # call model
        input_dim = np.expand_dims(X_train, axis = -1).shape[1:]
        output_dim = 1
        model_3d = ontram(img_model_linear(input_dim, output_dim))         
       
        # Define data loaders.
        
        train_loader = tf.data.Dataset.from_tensor_slices((X_train[:ntrain], y_train_enc[:ntrain]))
        validation_loader = tf.data.Dataset.from_tensor_slices((X_valid[:nvalid], y_valid_enc[:nvalid]))

        # data augmentation
        datagen = ImageDataGenerator(
            rotation_range=20,
            width_shift_range=0.2,
            height_shift_range=0.2,
            zoom_range=0.15,
            shear_range=0.15,
            fill_mode="nearest")
        datagen.fit(X_train[:ntrain])

        validation_dataset = (
            validation_loader.shuffle(len(X_valid[:nvalid]))
            .map(validation_preprocessing)
            .batch(batch_size)
            .prefetch(2)
        )


        #compile
        model_3d.compile(optimizer=keras.optimizers.Adam(learning_rate=5*1e-5),
                                loss=ontram_loss(C, batch_size),
                                metrics=[ontram_acc(C, batch_size)])      
        
        
        
        checkpoint_cb = keras.callbacks.ModelCheckpoint(
            filepath = os.path.join('/tf/notebooks/schnemau/xAI_stroke_3d/data/results_real/', f'model_ontram{j}_CIB_{i}.h5'),
            verbose = (1 if i == 0 and j == 0 else 0),
            save_weights_only = True,
            monitor = "val_loss", #'val_acc',
            mode = 'min',
            save_best_only = True)

        early_stopping_cb = keras.callbacks.EarlyStopping(monitor="val_loss", patience=100, restore_best_weights=True)
        
        # Train the model, doing validation at the end of each epoch
        if train_CIB_noreal:
            hist = model_3d.fit(
                datagen.flow(X_train[:ntrain], y_train_enc[:ntrain], batch_size=batch_size, shuffle=True),
                validation_data=validation_dataset,
                epochs=epochs,
                shuffle=True,
                verbose=(1 if i == 0 and j == 0 else 0),
                callbacks=[checkpoint_cb, early_stopping_cb]
            ) 
            pkl.dump(hist.history, open(os.path.join('/tf/notebooks/schnemau/xAI_stroke_3d/data/results_real/', f'model_{j}_ontram_CIB_{i}.pkl')), protocol=4)
            histplt = hist.history         
       
        
        end2 = time.time()
        print(" ")   
        print("Duration of Training: " + str(end2-start2))  
        
    end1 = time.time()
    print(" ")   
    print("Duration of Split: " + str(end1-start1))  
        
end0 = time.time()
print(" ")
print("Duration of Everything: " + str(end0-start0))  

## CI_B

In [ ]:
C = Y_train_MRS.shape[1]
batch_size = 5
epochs = 250
ntrain = 325
nvalid = 40
ntest = 40

In [ ]:
# function for preprocessing
def train_preprocessing(data, label):
    """Process training data."""
    volume = data
    volume = zoom(volume)
    volume = rotate(volume)
    volume = shift(volume)
    volume = flip(volume)
    return volume, label

# Define data sets for Jonas Model - no ordinal structure of 0,1
train_loader = tf.data.Dataset.from_tensor_slices((X_train[:ntrain], Y_train[:ntrain]))
validation_loader = tf.data.Dataset.from_tensor_slices((X_valid[:nvalid], Y_valid[:nvalid]))
test_loader = tf.data.Dataset.from_tensor_slices((X_test[:ntest], Y_test[:ntest]))

train_dataset = (train_loader.shuffle(len(X_train[:ntrain]))
                 .map(train_preprocessing)
                 .batch(batch_size, drop_remainder = True))

validation_dataset = (validation_loader.batch(batch_size, drop_remainder = True))
test_dataset = (test_loader.batch(len(X_test[:ntest])))

# Define data sets MRS for ordinal structure
train_loader_MRS = tf.data.Dataset.from_tensor_slices((X_train[:ntrain], Y_train_MRS[:ntrain]))
validation_loader_MRS = tf.data.Dataset.from_tensor_slices((X_valid[:nvalid], Y_valid_MRS[:nvalid]))
test_loader_MRS = tf.data.Dataset.from_tensor_slices((X_test[:ntest], Y_test_MRS[:ntest]))

train_dataset_MRS = (train_loader_MRS.shuffle(len(X_train[:ntrain]))
                 .map(train_preprocessing)
                 .batch(batch_size, drop_remainder = True))

validation_dataset_MRS = (validation_loader_MRS.batch(batch_size, drop_remainder = True))
test_dataset_MRS = (test_loader_MRS.batch(len(X_test[:ntest])))

In [ ]:
fit_CIB = True

In [ ]:
if fit_CIB:
    for i in range(5):
            mod_ontram_lin2 = ontram(img_model_linear(X_train.shape[1:], 1))
            mod_ontram_lin2.compile(optimizer=keras.optimizers.Adam(5*1e-5),
                                    loss=ontram_loss(C, batch_size),
                                    metrics=[ontram_acc(C, batch_size)])

            checkpoint = ModelCheckpoint(os.path.join('/tf/notebooks/schnemau/xAI_stroke_3d/ensembling_results/', f'model_{i}_ontram_CIB_fini.h5'),
                                        save_best_only=True, save_weights_only=True)
            #early_stopping = EarlyStopping(patience=8, restore_best_weights=True)

            mod_ontram_lin2.fit(train_dataset_MRS,
                                validation_data=validation_dataset_MRS,
                                epochs=250,
                                shuffle=True,
                                callbacks=checkpoint)



                                

## CI_B COMPLETE JONAS SET-UP

In [7]:
C = Y_train_MRS.shape[1]
batch_size = 5
epochs = 250
ntrain = 325
nvalid = 40
ntest = 40

In [8]:
def img_model_linear_final(input_shape, output_shape, activation = "linear"):
    initializer = keras.initializers.he_normal(seed = 2202)
    in_ = keras.Input(shape = input_shape)

    # conv block 0
    x = keras.layers.Convolution3D(32, kernel_size=(3, 3, 3), padding = 'same', activation = 'relu')(in_)
    x = keras.layers.MaxPooling3D(pool_size=(2, 2, 2))(x)
    # conv block 1
    x = keras.layers.Convolution3D(32, kernel_size=(3, 3, 3), padding = 'same', activation = 'relu')(x)
    x = keras.layers.MaxPooling3D(pool_size=(2, 2, 2))(x)
    # conv block 2
    x = keras.layers.Convolution3D(64, kernel_size=(3, 3, 3), padding = 'same', activation = 'relu')(x)
    x = keras.layers.MaxPooling3D(pool_size=(2, 2, 2))(x)
    # conv block 3
    x = keras.layers.Convolution3D(64, kernel_size=(3, 3, 3), padding = 'same', activation = 'relu')(x)
    x = keras.layers.MaxPooling3D(pool_size=(2, 2, 2))(x)

    # cnn to flat connection
    x = layers.GlobalAveragePooling3D()(x) 
    
    # flat block
    x = keras.layers.Dense(128, activation = 'relu')(x)
    x = keras.layers.Dropout(0.3)(x)
    x = keras.layers.Dense(128, activation = 'relu')(x)
    x = keras.layers.Dropout(0.3)(x)
    out_ = keras.layers.Dense(output_shape, activation = activation, use_bias = False)(x) 
    nn_im = keras.Model(inputs = in_, outputs = out_)
    return nn_im

In [9]:
def validation_preprocessing(volume, label):
    """Process validation data by only adding a channel."""
    volume = tf.expand_dims(volume, axis=3)
    return volume, label

In [ ]:
# call model
input_dim = np.expand_dims(X_train, axis = -1).shape[1:]
#input_dim = np.expand_dims(X_train, axis = -1).shape[1:5]
output_dim = C-1
model_3d = ontram(img_model_linear_final(input_dim, output_dim))         

# Define data loaders.
train_loader = tf.data.Dataset.from_tensor_slices((X_train[:ntrain], Y_train_MRS[:ntrain]))
validation_loader = tf.data.Dataset.from_tensor_slices((X_valid[:nvalid], Y_valid_MRS[:nvalid]))

# data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.15,
    shear_range=0.15,
    fill_mode="nearest")
datagen.fit(X_train[:ntrain])

validation_dataset = (
    validation_loader.shuffle(len(X_valid[:nvalid]))
    .map(validation_preprocessing)
    .batch(batch_size)
    .prefetch(2)
)

#compile
model_3d.compile(optimizer=keras.optimizers.Adam(learning_rate=5*1e-5),
                        loss=ontram_loss(C, batch_size),
                        metrics=[ontram_acc(C, batch_size)])
        

In [ ]:
train_CIB_noreal = True

In [ ]:
model_3d.fit(
        datagen.flow(X_train[:ntrain], Y_train_MRS[:ntrain], batch_size=batch_size, shuffle=True),
        validation_data=validation_dataset,
        epochs=250,
        shuffle=True,
        verbose=1,
        steps_per_epoch = 100)

In [ ]:
checkpoint_cb = keras.callbacks.ModelCheckpoint(
    filepath = os.path.join('/tf/notebooks/schnemau/xAI_stroke_3d/data/results_real/', f'iniCIB_1.h5'),
    verbose = 1,
    save_weights_only = True,
    monitor = "val_loss", #'val_acc',
    mode = 'min',
    save_best_only = True)

early_stopping_cb = keras.callbacks.EarlyStopping(monitor="val_loss", patience=100, restore_best_weights=True)

# Train the model, doing validation at the end of each epoch
if train_CIB_noreal:
    hist = model_3d.fit(
        datagen.flow(X_train[:ntrain], Y_train_MRS[:ntrain], batch_size=batch_size, shuffle=True),
        validation_data=validation_dataset,
        epochs=250,
        shuffle=True,
        verbose=1,
        steps_per_epoch = 150,        
        callbacks=[checkpoint_cb, early_stopping_cb]) 
    pkl.dump(hist.history, open(os.path.join('/tf/notebooks/schnemau/xAI_stroke_3d/data/results_real/', f'iniCIB_1.pkl')), protocol=4)
    histplt = hist.history 

In [10]:
for i in range(5):
    # call model
    input_dim = np.expand_dims(X_train, axis = -1).shape[1:]
    #input_dim = np.expand_dims(X_train, axis = -1).shape[1:5]
    output_dim = C-1
    model_3d = ontram(img_model_linear_final(input_dim, output_dim))         

    # Define data loaders.
    train_loader = tf.data.Dataset.from_tensor_slices((X_train[:ntrain], Y_train_MRS[:ntrain]))
    validation_loader = tf.data.Dataset.from_tensor_slices((X_valid[:nvalid], Y_valid_MRS[:nvalid]))

    # data augmentation
    datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        zoom_range=0.15,
        shear_range=0.15,
        fill_mode="nearest")
    datagen.fit(X_train[:ntrain])

    validation_dataset = (
        validation_loader.shuffle(len(X_valid[:nvalid]))
        .map(validation_preprocessing)
        .batch(batch_size)
        .prefetch(2)
    )

    #compile
    model_3d.compile(optimizer=keras.optimizers.Adam(learning_rate=5*1e-5),
                            loss=ontram_loss(C, batch_size),
                            metrics=[ontram_acc(C, batch_size)])
    
    checkpoint_cb = keras.callbacks.ModelCheckpoint(
    filepath = os.path.join('/tf/notebooks/schnemau/xAI_stroke_3d/data/results_real/', f'CIB_{i}_.h5'),
    verbose = 1,
    save_weights_only = True,
    monitor = "val_loss", #'val_acc',
    mode = 'min',
    save_best_only = True)

    early_stopping_cb = keras.callbacks.EarlyStopping(monitor="val_loss", patience=100, restore_best_weights=True)
    
    hist = model_3d.fit(
        datagen.flow(X_train[:ntrain], Y_train_MRS[:ntrain], batch_size=batch_size, shuffle=True),
        validation_data=validation_dataset,
        epochs=250,
        shuffle=True,
        verbose=1,
        steps_per_epoch = 1000,        
        callbacks=[checkpoint_cb, early_stopping_cb])
    
    #pkl.dump(hist.history, open(os.path.join('/tf/notebooks/schnemau/xAI_stroke_3d/data/results_real/', f'CIB_{i}_.pkl')), protocol=4)


/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:940: UserWarning: Expected input to be images (as Numpy array) following the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3 or 4 channels on axis 3. However, it was passed an array with shape (325, 128, 128, 28) (28 channels).
  ' channels).')
/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/numpy_array_iterator.py:127: UserWarning: NumpyArrayIterator is set to use the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3, or 4 channels on axis 3. However, it was passed an array with shape (325, 128, 128, 28) (28 channels).
  str(self.x.shape[channels_axis]) + ' channels).')


Epoch 1/250
1000/1000 [==============================] - ETA: 0s - loss: 0.5055 - acc: 0.8040
Epoch 00001: val_loss improved from inf to 0.46902, saving model to /tf/notebooks/schnemau/xAI_stroke_3d/data/results_real/CIB_0_.h5
1000/1000 [==============================] - 115s 115ms/step - loss: 0.5055 - acc: 0.8040 - val_loss: 0.4690 - val_acc: 0.8250
Epoch 2/250
1000/1000 [==============================] - ETA: 0s - loss: 0.4908 - acc: 0.8100
Epoch 00002: val_loss improved from 0.46902 to 0.46834, saving model to /tf/notebooks/schnemau/xAI_stroke_3d/data/results_real/CIB_0_.h5
1000/1000 [==============================] - 114s 114ms/step - loss: 0.4908 - acc: 0.8100 - val_loss: 0.4683 - val_acc: 0.8250
Epoch 3/250
1000/1000 [==============================] - ETA: 0s - loss: 0.4796 - acc: 0.8098
Epoch 00003: val_loss did not improve from 0.46834
1000/1000 [==============================] - 115s 115ms/step - loss: 0.4796 - acc: 0.8098 - val_loss: 0.4719 - val_acc: 0.8250
Epoch 4/250
1000

Epoch 30/250
1000/1000 [==============================] - ETA: 0s - loss: 0.0109 - acc: 0.9972
Epoch 00030: val_loss did not improve from 0.43137
1000/1000 [==============================] - 116s 116ms/step - loss: 0.0109 - acc: 0.9972 - val_loss: 1.9978 - val_acc: 0.7250
Epoch 31/250
1000/1000 [==============================] - ETA: 0s - loss: 0.0096 - acc: 0.9978
Epoch 00031: val_loss did not improve from 0.43137
1000/1000 [==============================] - 117s 117ms/step - loss: 0.0096 - acc: 0.9978 - val_loss: 2.0046 - val_acc: 0.7000
Epoch 32/250
1000/1000 [==============================] - ETA: 0s - loss: 0.0118 - acc: 0.9964
Epoch 00032: val_loss did not improve from 0.43137
1000/1000 [==============================] - 116s 116ms/step - loss: 0.0118 - acc: 0.9964 - val_loss: 1.9657 - val_acc: 0.7500
Epoch 33/250
1000/1000 [==============================] - ETA: 0s - loss: 9.6859e-04 - acc: 1.0000
Epoch 00033: val_loss did not improve from 0.43137
1000/1000 [====================

Epoch 60/250
1000/1000 [==============================] - ETA: 0s - loss: 0.0064 - acc: 0.9980
Epoch 00060: val_loss did not improve from 0.43137
1000/1000 [==============================] - 115s 115ms/step - loss: 0.0064 - acc: 0.9980 - val_loss: 2.5228 - val_acc: 0.7000
Epoch 61/250
1000/1000 [==============================] - ETA: 0s - loss: 0.0030 - acc: 0.9990
Epoch 00061: val_loss did not improve from 0.43137
1000/1000 [==============================] - 115s 115ms/step - loss: 0.0030 - acc: 0.9990 - val_loss: 2.4199 - val_acc: 0.7250
Epoch 62/250
1000/1000 [==============================] - ETA: 0s - loss: 1.4774e-04 - acc: 1.0000
Epoch 00062: val_loss did not improve from 0.43137
1000/1000 [==============================] - 116s 116ms/step - loss: 1.4774e-04 - acc: 1.0000 - val_loss: 2.7929 - val_acc: 0.7250
Epoch 63/250
1000/1000 [==============================] - ETA: 0s - loss: 9.2523e-05 - acc: 1.0000
Epoch 00063: val_loss did not improve from 0.43137
1000/1000 [============

Epoch 90/250
1000/1000 [==============================] - ETA: 0s - loss: 2.3825e-05 - acc: 1.0000
Epoch 00090: val_loss did not improve from 0.43137
1000/1000 [==============================] - 115s 115ms/step - loss: 2.3825e-05 - acc: 1.0000 - val_loss: inf - val_acc: 0.7250
Epoch 91/250
1000/1000 [==============================] - ETA: 0s - loss: 1.0151e-05 - acc: 1.0000
Epoch 00091: val_loss did not improve from 0.43137
1000/1000 [==============================] - 115s 115ms/step - loss: 1.0151e-05 - acc: 1.0000 - val_loss: inf - val_acc: 0.7250
Epoch 92/250
1000/1000 [==============================] - ETA: 0s - loss: 0.0193 - acc: 0.9948
Epoch 00092: val_loss did not improve from 0.43137
1000/1000 [==============================] - 116s 116ms/step - loss: 0.0193 - acc: 0.9948 - val_loss: 2.2909 - val_acc: 0.7250
Epoch 93/250
1000/1000 [==============================] - ETA: 0s - loss: 2.6166e-04 - acc: 1.0000
Epoch 00093: val_loss did not improve from 0.43137
1000/1000 [==========

FileNotFoundError: [Errno 2] No such file or directory: '/tf/notebooks/schnemau/xAI_stroke_3d/data/results_real/CIB_0_.pkl'

## CI_B & LS_X

In [ ]:
# function for preprocessing
def train_preprocessing2(data, label):
    """Process training data."""
    volume = data[0] # intercept: image
    tabular = data[1] # shift: tabular
    volume = zoom(volume)
    volume = rotate(volume)
    volume = shift(volume)
    volume = flip(volume)
    return (volume, tabular), label

# Define data sets
train_data_2 = tf.data.Dataset.from_tensor_slices((X_train[:ntrain], X_tab_train[:ntrain]))
train_labels_2 = tf.data.Dataset.from_tensor_slices((Y_train_MRS[:ntrain]))

valid_data_2 = tf.data.Dataset.from_tensor_slices((X_valid[:ntrain], X_tab_valid[:ntrain]))
valid_labels_2 = tf.data.Dataset.from_tensor_slices((Y_valid_MRS[:nvalid]))

test_data_2 = tf.data.Dataset.from_tensor_slices((X_test[:ntest], X_tab_test[:ntest]))
test_labels_2 = tf.data.Dataset.from_tensor_slices((Y_test_MRS[:ntest]))

train_loader_2 = tf.data.Dataset.zip((train_data_2, train_labels_2))
validation_loader_2 = tf.data.Dataset.zip((valid_data_2, valid_labels_2))
test_loader_2 = tf.data.Dataset.zip((test_data_2, test_labels_2))

train_dataset_MRS_2 = (train_loader_2.shuffle(ntrain)
                 .map(train_preprocessing2)
                 .batch(batch_size))

validation_dataset_MRS_2 = (validation_loader_2.batch(batch_size))

test_dataset_MRS_2 = (test_loader_2.batch(len(X_test[:ntest])))

In [ ]:
fit_CIBLSX = True
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
if fit_CIBLSX:
    for i in range(5):
        CI_B = img_model_linear(X_train.shape[1:], 1)
        LS_X = mod_linear_shift(X_tab_train.shape[1])
        mod_ontram_CIBLSX = ontram(CI_B, LS_X)
        mod_ontram_CIBLSX.compile(optimizer = keras.optimizers.Adam(),
                    loss = ontram_loss(C, batch_size),
                    metrics = [ontram_acc(C, batch_size)])    

        checkpoint2 = ModelCheckpoint(os.path.join('/tf/notebooks/schnemau/xAI_stroke_3d/ensembling_results/', f'model_{i}_{i}_ontram_CIBLSX_.h5'),
                                        save_best_only=True, save_weights_only=True)
        
        early_stopping2 = EarlyStopping(patience=100, restore_best_weights=True)
        
        
        mod_ontram_CIBLSX.fit(train_dataset_MRS_2, 
                validation_data = validation_dataset_MRS_2,
                epochs = 250,
                shuffle = True,
                callbacks=[checkpoint2, early_stopping2])

## Jonas

In [ ]:
train_jonas = False

In [ ]:
if train_jonas:
        for i in range(5):
                jon_softmax = stroke_binary_3d_softmax()
                jon_softmax.compile(loss=tf.keras.losses.categorical_crossentropy,
                optimizer=keras.optimizers.Adam(learning_rate=5*1e-5),
                metrics=["acc", tf.keras.metrics.AUC()])   

                checkpoint3 = ModelCheckpoint(os.path.join('/tf/notebooks/schnemau/xAI_stroke_3d/ensembling_results/', f'model_jonas_{i}.h5'),
                                        save_best_only=True, save_weights_only=True)
        
                early_stopping3 = EarlyStopping(patience=7, restore_best_weights=True)  

                jon_softmax.fit(train_dataset_MRS, 
                    validation_data = validation_dataset_MRS,
                    epochs = 35,
                    shuffle = True,
                    callbacks=[checkpoint3, early_stopping3])

# Jonas from earlier check

In [ ]:
path = '/tf/notebooks/schnemau/xAI_stroke_3d/weights/andrea_split/3d_cnn_binary_model_split6_unnormalized_avg_layer_paper_model_sigmoid_activation_14.h5'

input_dim = X_train.shape[1:]
output_dim = 1

layer_connection = "globalAveragePooling"
last_activation = "sigmoid"

# call model
model_3d2 = md.stroke_binary_3d(input_dim = input_dim,
                               output_dim = output_dim,
                               layer_connection = layer_connection,
                               last_activation = last_activation)
model_3d2.compile(
    loss="binary_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=5*1e-5),
    metrics=["acc", tf.keras.metrics.AUC()]
)

model_3d2.load_weights(path)
